In [1]:
import json
import pandas as pd
import time
import datetime
from collections import defaultdict
from sql_queries import events_table_insert, teams_table_insert, players_table_insert, matches_table_insert
from create_tables import create_database, drop_tables, create_tables
from collections import Counter

In [2]:
def find_common_keys(json_dicts):
    high_level_keys = Counter()
    dict_count = len(json_dicts)
    for d in json_dicts:
        high_level_keys.update(d.keys())
        
    # identify columns appearing in at least half of data dictionaries
    common_keys = [key for key in high_level_keys.keys() if high_level_keys[key] == dict_count]
    
    return common_keys

In [3]:
with open('./data/15946.json', 'r+') as f:
    match_events = json.load(f)

In [4]:
common_keys = find_common_keys(match_events)

In [5]:
common_keys

['id',
 'index',
 'period',
 'timestamp',
 'minute',
 'second',
 'type',
 'possession',
 'possession_team',
 'play_pattern',
 'team']

In [6]:
cur, conn = create_database()

drop_tables(cur, conn)
create_tables(cur, conn)

In [7]:
def def_value():
    return None

In [8]:
with open('./data/15946.json', 'r+') as f:
    match_events = json.load(f)
    
for event in match_events:
    events_table_dict = defaultdict(def_value)
    teams_table_dict = defaultdict(def_value)
    players_table_dict = defaultdict(def_value)
#     play_patterns_table_dict = defaultdict(def_value)
#     related_events_table_dict = defaultdict(def_value)
#     matches_table_dict = defaultdict(def_value)
    
    # TODO: assing the match ID to the events_table_dict and matches_table_dict
    events_table_dict["match"] = 15946
#     matches_table_dict["id"] = 15946
    
    
    
    for key, value in event.items():
        if key == "id":
            events_table_dict["id"] = value
            
        elif key == "index":
            events_table_dict["index"] = value
            
        elif key == "type":
            events_table_dict["type"] = value["name"]
            
        elif key == "play_pattern":
            events_table_dict["play_pattern"] = value["name"]
            
        elif key == "timestamp":
            events_table_dict["event_time"] = value
            
        elif key == "period":
            events_table_dict["period"] = value
            
        elif key == "location":
            location_x, location_y = value
            
            events_table_dict["location_x"] = location_x,
            events_table_dict["location_y"] = location_y
            
        elif key == "possession_team":
            events_table_dict["possession_team"] = value["id"]
            
        elif key == "possession":
            events_table_dict["possession"] = value
            
        elif key == "player":
            events_table_dict["player"] = value["id"]
            
            players_table_dict["id"] = value["id"]
            players_table_dict["name"] = value["name"]
            
        elif key == "team":
            events_table_dict["team"] = value["id"]
            
            teams_table_dict["id"] = value["id"]
            teams_table_dict["name"] = value["name"]
            
            players_table_dict["team"] = value["id"]
            
        elif key == "duration":
            events_table_dict["duration"] = value
            
        elif key == "position":
            players_table_dict["position"] = value["name"]
            
    cur.execute(
        events_table_insert,
        (
            events_table_dict["id"],
            events_table_dict["index"],
            events_table_dict["type"],
            events_table_dict["play_pattern"],
            events_table_dict["event_time"],
            events_table_dict["period"],
            events_table_dict["location_x"],
            events_table_dict["location_y"],
            events_table_dict["possession_team"],
            events_table_dict["possession"],
            events_table_dict["team"],
            events_table_dict["player"],
            events_table_dict["team"],
            events_table_dict["duration"],
        )
    )
    
    cur.execute(
        teams_table_insert,
        (
            teams_table_dict["id"],
            teams_table_dict["name"],
            teams_table_dict["gender"],
            teams_table_dict["country"]
        )
    )
    
    if players_table_dict["id"]:
        cur.execute(
            players_table_insert,
            (
                players_table_dict["id"],
                players_table_dict["name"],
                players_table_dict["position"],
                players_table_dict["team"]
            )
        )
conn.commit()

In [9]:
conn.close()